## Import libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import numpy as np
from numpy import polyfit
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Conv1D, MaxPooling1D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Flatten
import tensorflow as tf
import plotly.express as px
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Import And Preprocess Data Set

In [4]:
df = pd.read_csv("/content/drive/My Drive/CE397Proj/CO2/FinalCo2.csv")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
# Convert Effective date type to datetime
df["Date"] = pd.to_datetime(df["Date"])
# df['DATE'] = pd.to_datetime(df['DATE']).dt.tz_localize(None)
df.head(2)

,Date,Value
0,1959-03-01,316.65
1,1959-04-01,317.72


In [5]:
df.tail()

,Date,Value
777,2023-12-01,421.86
778,2024-01-01,422.80
779,2024-02-01,424.62
780,2024-03-01,425.38
781,2024-04-01,426.57


In [6]:
df=df.loc[df['Date']>='2000-01-01']

In [7]:
df.set_index('Date', inplace=True)

df.head()

,Value
Date,
2000-01-01,369.45
2000-02-01,369.71
2000-03-01,370.75
2000-04-01,371.98
2000-05-01,371.74


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 292 entries, 2000-01-01 to 2024-04-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Value   292 non-null    float64
dtypes: float64(1)
memory usage: 4.6 KB


In [9]:
train = df[:190]
valid = df[190:250]
test = df[250:]

scaler = MinMaxScaler()

# Fit scaler on training data and transform all subsets
train_data = scaler.fit_transform(train[['Value']])
validate_data = scaler.transform(valid[['Value']])
test_data = scaler.transform(test[['Value']])

print("Train data size:", len(train_data))
print("Validation data size:", len(validate_data))
print("Test data size:", len(test_data))


Train data size: 190
Validation data size: 60
Test data size: 42


In [10]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Bidirectional, LSTM, Dropout, Concatenate
from tensorflow.keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Define the sequence creation function
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

# Create sequences
seq_length = 5  # You can experiment with this value
X_train, y_train = create_sequences(train_data, seq_length)
X_val, y_val = create_sequences(validate_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Reshape for CNN and LSTM
X_train_cnn = X_train.reshape(-1, seq_length, 1)
X_val_cnn = X_val.reshape(-1, seq_length, 1)
X_test_cnn = X_test.reshape(-1, seq_length, 1)

X_train_lstm = X_train.reshape(-1, seq_length, 1)
X_val_lstm = X_val.reshape(-1, seq_length, 1)
X_test_lstm = X_test.reshape(-1, seq_length, 1)

### Implemented Code

In [11]:
def create_train_hybrid_model_grid_search(activation_function, optimizer_name):
    """
    Creates, compiles, and trains a CNN model with the specified activation function and optimizer.

    Args:
        activation_function: The activation function to use in the model layers.
        optimizer_name: The name of the optimizer to use ('adam', 'adamw', or 'adamax').

    Returns:
        A tuple containing the trained Keras model and its training history.
    """
   # CNN-LSTM hybrid model
    # Input shape for CNN
    cnn_input = Input(shape=(seq_length, 1))
    x = Conv1D(filters=64, kernel_size=3, activation=activation_function, kernel_regularizer=l2(0.01))(cnn_input)  # L2 regularization
    x = MaxPooling1D(pool_size=2)(x)
    x = Flatten()(x)

    # LSTM Model
    lstm_input = Input(shape=(seq_length, 1))
    y = Bidirectional(LSTM(200, return_sequences=True, kernel_regularizer=l2(0.01)))(lstm_input)
    y = Dropout(0.5)(y)  # Higher dropout for regularization
    y = Bidirectional(LSTM(200, kernel_regularizer=l2(0.01)))(y)
    y = Dropout(0.5)(y)
    y = Dense(100, activation=activation_function)(y)

    # Combining CNN and LSTM outputs
    combined = Concatenate()([x, y])
    combined = Dense(100, activation=activation_function)(combined)
    combined = Dense(1)(combined)  # Final output layer

    # Create the model
    model = Model(inputs=[cnn_input, lstm_input], outputs=combined)

    if optimizer_name == 'adam':
        optimizer = optimizers.Adam()
    elif optimizer_name == 'adamw':
        optimizer = optimizers.AdamW()
    elif optimizer_name == 'adamax':
        optimizer = optimizers.Adamax()
    else:
        raise ValueError(f"Unsupported optimizer: {optimizer_name}. Choose from 'adam', 'adamw', or 'adamax'.")

    # Compile the model
    model.compile(optimizer=optimizer_name, loss='mean_squared_error')

    # Set up early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(
        [X_train_cnn, X_train_lstm],
        y_train,
        epochs=100,
        batch_size=32,
        validation_data=([X_val_cnn, X_val_lstm], y_val),
        callbacks=[early_stopping],
        verbose=0  # Suppress epoch-by-epoch output
    )

    # Evaluate the model
    loss = model.evaluate([X_test_cnn, X_test_lstm], y_test, verbose=0) # Suppress evaluation output
    return model, history

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from numpy import sqrt

activation_functions = ['sigmoid', 'tanh', 'relu', 'leaky_relu', 'silu', 'gelu']
optimizers_list = ['adam', 'adamw', 'adamax']
num_trials = 5 # Number of trials for each combination
grid_search_results = {}

for activation in activation_functions:
    grid_search_results[activation] = {}
    for optimizer_name in optimizers_list:
        grid_search_results[activation][optimizer_name] = []

for trial in range(num_trials):
    print(f"--- Starting Grid Search Trial {trial + 1}/{num_trials} ---")
    for activation in activation_functions:
        for optimizer_name in optimizers_list:
            print(f"Training model with {activation} activation and {optimizer_name} optimizer for Trial {trial + 1}...")
            model, history = create_train_hybrid_model_grid_search(activation, optimizer_name)

            # Evaluate the model
            test_predict = model.predict([X_test_cnn, X_test_lstm])
            test_predict = scaler.inverse_transform(test_predict)
            y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

            test_mse = mean_squared_error(y_test_inv, test_predict)
            test_rmse = sqrt(test_mse)
            test_mae = mean_absolute_error(y_test_inv, test_predict)

            # Calculate average test error for this trial and combination
            average_error = (test_mse + test_rmse + test_mae) / 3

            # Store the result
            grid_search_results[activation][optimizer_name].append(average_error)

            print(f"Finished training and evaluating model with {activation} activation and {optimizer_name} optimizer for Trial {trial + 1}.")

print("\n--- Grid Search completed ---")

--- Starting Grid Search Trial 1/5 ---
Training model with sigmoid activation and adam optimizer for Trial 1...
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 739ms/step
Finished training and evaluating model with sigmoid activation and adam optimizer for Trial 1.
Training model with sigmoid activation and adamw optimizer for Trial 1...
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 721ms/step
Finished training and evaluating model with sigmoid activation and adamw optimizer for Trial 1.
Training model with sigmoid activation and adamax optimizer for Trial 1...


1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 719ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 744ms/step
Finished training and evaluating model with sigmoid activation and adamax optimizer for Trial 1.
Training model with tanh activation and adam optimizer for Trial 1...
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 805ms/step
Finished training and evaluating model with tanh activation and adam optimizer for Trial 1.
Training model with tanh activation and adamw optimizer for Trial 1...
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 725ms/step
Finished training and evaluating model with tanh activation and adamw optimizer for Trial 1.
Training model with tanh activation and adamax optimizer for Trial 1...
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 674ms/step
Finished training and evaluating model with tanh activation and adamax optimizer for Trial 1.
Training model with relu activation and adam optimizer for Trial 1...
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 636ms/step
Finished training and evaluating model with relu activation and adam optimizer for Trial 1.
Training model with relu activation and adamw optimize

In [ ]:
average_grid_search_results = {}
for activation in activation_functions:
    average_grid_search_results[activation] = {}
    for optimizer_name in optimizers_list:
        average_error = sum(grid_search_results[activation][optimizer_name]) / num_trials
        average_grid_search_results[activation][optimizer_name] = average_error

# Find the best combination
best_combination = None
min_average_error = float('inf')

for activation, optimizer_results in average_grid_search_results.items():
    for optimizer_name, average_error in optimizer_results.items():
        if average_error < min_average_error:
            min_average_error = average_error
            best_combination = (activation, optimizer_name)

print("Average error for each combination across all trials:")
for activation, optimizer_results in average_grid_search_results.items():
    for optimizer_name, average_error in optimizer_results.items():
        print(f"  {activation} + {optimizer_name}: {average_error:.4f}")

print(f"\nThe best combination of activation function and optimizer is: {best_combination}")
print(f"With an average error of: {min_average_error:.4f}")

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from numpy import sqrt

# Train the final model using the best combination
best_activation, best_optimizer = best_combination
best_model, best_history = create_train_hybrid_model_grid_search(best_activation, best_optimizer)

# Evaluate the best model
test_predict_best = best_model.predict([X_test_cnn, X_test_lstm])

# Inverse transform the predictions and actual values
test_predict_best_inv = scaler.inverse_transform(test_predict_best)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate performance metrics for the best model
best_test_mse = mean_squared_error(y_test_inv, test_predict_best_inv)
best_test_rmse = sqrt(best_test_mse)
best_test_mae = mean_absolute_error(y_test_inv, test_predict_best_inv)
best_test_r2 = r2_score(y_test_inv, test_predict_best_inv)

# Print the metrics for the best model
print("Best Model Performance Metrics:")
print(f"  Test MSE: {best_test_mse:.4f}")
print(f"  Test RMSE: {best_test_rmse:.4f}")
print(f"  Test MAE: {best_test_mae:.4f}")
print(f"  Test R²: {best_test_r2:.4f}")

# Plotting
plt.figure(figsize=(15, 6), dpi=150)

# Plot actual values
plt.plot(df.index, df['Value'], label='Actual', color='blue')

# Generate and plot training predictions for the best model
train_predict_best = best_model.predict([X_train_cnn, X_train_lstm])
train_predict_best_inv = scaler.inverse_transform(train_predict_best)
# Adjust the index for training predictions
plt.plot(df.index[seq_length : len(train) ], train_predict_best_inv.flatten(), label='Train Predictions', color='green')

# Generate and plot validation predictions for the best model
val_predict_best = best_model.predict([X_val_cnn, X_val_lstm])
val_predict_best_inv = scaler.inverse_transform(val_predict_best)
# Adjust the index for validation predictions
plt.plot(df.index[len(train) + seq_length : len(train) + len(valid)], val_predict_best_inv.flatten(), label='Validation Predictions', color='red')


# Plot test predictions (already calculated)
# Adjust the index for test predictions
plt.plot(df.index[len(train) + len(valid) + seq_length : len(df)], test_predict_best_inv.flatten(), label='Test Predictions', color='orange')

plt.title(f'CO2 Emission Forecasting (Best Hybrid Model with {best_activation} activation and {best_optimizer} optimizer)')
plt.xlabel('Date')
plt.ylabel('CO2 Values')
plt.legend(loc='upper left')
plt.show()